## 파일 업로드
#### 1부터 30 :train 폴더 만들기, 31부터 40: test 폴더 만들기 (각자 경로 불러오기)

In [1]:
import glob
import pandas as pd
import numpy as np

file_paths = [r"C:\Users\호재\Desktop\대학\3학년 2학기\기계학습론\data\IC_PBL\abnormal\train",\
             r"C:\Users\호재\Desktop\대학\3학년 2학기\기계학습론\data\IC_PBL\abnormal\test",\
             r"C:\Users\호재\Desktop\대학\3학년 2학기\기계학습론\data\IC_PBL\normal\train",\
             r"C:\Users\호재\Desktop\대학\3학년 2학기\기계학습론\data\IC_PBL\normal\test"]

dataframes_train_abn = []
dataframes_test_abn  = []
dataframes_train_n  = []
dataframes_test_n  = []

df_list = [dataframes_train_abn, dataframes_test_abn, dataframes_train_n, dataframes_test_n]

for i in range(len(file_paths)) :
    files = glob.glob(file_paths[i] + "\\**\\*.csv", recursive=True)
    
    for file_path in files:
        df = pd.read_csv(file_path, encoding='cp949', header=None)
        df_list[i].append(df[1])


## 중복 정도에 따라 데이터 분할

In [2]:
dataframes_train_abn_0 = []
dataframes_test_abn_0 = []
dataframes_train_n_0  = []
dataframes_test_n_0 = []

df_list_0 = [dataframes_train_abn_0, dataframes_test_abn_0, dataframes_train_n_0, dataframes_test_n_0]

for j in range(len(df_list)):
    df_list_0[j].extend([df[i:i+256] for df in df_list[j] for i in range(0, df.shape[0], 256)])


In [3]:
dataframes_train_abn_25 = []
dataframes_test_abn_25 = []
dataframes_train_n_25  = []
dataframes_test_n_25 = []

df_list_25 = [dataframes_train_abn_25, dataframes_test_abn_25, dataframes_train_n_25, dataframes_test_n_25]

for j in range(len(df_list)):
    df_list_25[j].extend([df[i:i+256] for df in df_list[j] for i in range(0, df.shape[0]-64, 256-64)])

In [4]:
dataframes_train_abn_50 = []
dataframes_test_abn_50 = []
dataframes_train_n_50  = []
dataframes_test_n_50 = []

df_list_50 = [dataframes_train_abn_50, dataframes_test_abn_50, dataframes_train_n_50, dataframes_test_n_50]

for j in range(len(df_list)):
    df_list_50[j].extend([df[i:i+256] for df in df_list[j] for i in range(0, df.shape[0]-128, 256-128)])

## Feature 추가

In [5]:
from scipy.stats import skew
from scipy.stats import kurtosis

def abs_mean(x):
    return np.mean(np.abs(x))
def abs_std(x) :
    return  np.std(np.abs(x))

def calculate_outlier_count(df):
    Q1 = df.quantile(0.25)
    Q3 = df.quantile(0.75)
    IQR = Q3 - Q1
    outliers = ((df < (Q1 - 1.0 * IQR)) | (df > (Q3 + 1.0 * IQR))).sum()
    return outliers


def m_average(df) :
    window_size = 4
    moving_average = df.rolling(window=window_size).mean()
    moving_average = moving_average.dropna(axis = 0)
    
    return(abs_mean(moving_average))

def abs_median(x) :
    return np.median(np.abs(x))

# 함수 정의
def calculate_summary_statistics(df_list):

    summary_data = []
    for df in df_list:
    
        summary_data.append({
            'min': df.min().tolist(),
            'max': df.max().tolist(),
            'mean': df.mean().tolist(),
            'abs_mean' : abs_mean(df).tolist(),
            'std': df.std().tolist(),
            'abs_std' : abs_std(df).tolist(),
            'median': df.median().tolist(),
            'abs_median': abs_median(df).tolist(),
            'skew': df.skew().tolist(),
            'kurt': df.kurtosis().tolist(),
            'outlier': calculate_outlier_count(df).tolist(),
            'M_average': m_average(df).tolist(),
            
    })

    return summary_data


## 최종 데이터셋

In [6]:
import pandas as pd

def process_df_list(df_list):
    result_list = []
    
    for i, df in enumerate(df_list):
        result_df = calculate_summary_statistics(df)
        result_df = pd.DataFrame(result_df)
        
        if i < 2 :
            result_df['target'] = 1  # 1: 불량
        else:
            result_df['target'] = 0

        result_list.append(result_df)
    
 
    train_df = pd.concat([result_list[i] for i in range(len(result_list)) if i % 2 == 0], ignore_index=True)
    test_df = pd.concat([result_list[i] for i in range(len(result_list)) if i % 2 == 1], ignore_index=True)

    return train_df, test_df


train_0, test_0 = process_df_list(df_list_0)

train_25, test_25 = process_df_list(df_list_25)

train_50, test_50 = process_df_list(df_list_50)

In [7]:
train_0

,min,max,mean,abs_mean,std,abs_std,median,abs_median,skew,kurt,outlier,M_average,target
0,-0.71970,0.50853,0.001912,0.228711,0.284993,0.169111,0.04794,0.18235,-0.481730,-0.447660,6,0.145982,1
1,-0.68132,0.50853,-0.002736,0.227290,0.283192,0.168022,0.04794,0.18235,-0.408038,-0.459732,8,0.143973,1
2,-0.68132,0.54691,0.000113,0.224891,0.278707,0.163701,0.02875,0.18235,-0.419592,-0.481267,3,0.145982,1
3,-0.68132,0.50853,0.000712,0.231562,0.287722,0.169825,0.04794,0.20147,-0.484267,-0.455853,8,0.150117,1
4,-0.68252,0.54571,-0.001086,0.224760,0.278707,0.163884,0.02755,0.18355,-0.419600,-0.481266,9,0.146006,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
25031,-0.45234,0.43045,0.001349,0.171114,0.197879,0.098617,0.00825,0.16177,-0.060824,-1.019075,1,0.152071,0
25032,-0.41223,0.47056,0.000074,0.173409,0.199613,0.098078,0.00997,0.18194,0.037731,-1.089629,0,0.153837,0
25033,-0.45062,0.43217,0.003073,0.171154,0.197880,0.098587,0.00997,0.16350,-0.060829,-1.019092,0,0.152104,0
25034,-0.37385,0.43217,0.000974,0.170188,0.197298,0.099053,0.00997,0.16350,0.060084,-1.081446,0,0.150689,0


In [8]:
test_50

,min,max,mean,abs_mean,std,abs_std,median,abs_median,skew,kurt,outlier,M_average,target
0,-0.41261,0.54695,0.000750,0.153004,0.188178,0.108916,0.009600,0.124740,0.170245,-0.287144,8,0.091518,1
1,-0.56614,0.50856,0.001349,0.152480,0.189399,0.111729,0.009600,0.124740,0.128140,-0.232756,9,0.093832,1
2,-0.56614,0.47018,-0.002849,0.153979,0.191747,0.113674,-0.009595,0.124740,0.016414,-0.124516,10,0.097018,1
3,-0.56614,0.47018,-0.001949,0.155853,0.192044,0.111581,0.009600,0.124740,0.065205,-0.311668,11,0.098004,1
4,-0.56614,0.50856,0.002699,0.153229,0.187999,0.108321,0.009600,0.143930,0.076127,-0.275791,10,0.094704,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
14982,-0.56587,0.47044,0.000412,0.216204,0.251373,0.127270,0.009860,0.201770,-0.031155,-1.002888,0,0.159639,0
14983,-0.56587,0.47044,0.000862,0.220929,0.257374,0.131049,0.009860,0.220440,-0.069040,-1.058081,0,0.161956,0
14984,-0.52749,0.47044,-0.000637,0.217408,0.251320,0.125097,0.009860,0.211105,-0.036667,-1.152032,0,0.156497,0
14985,-0.52749,0.54721,-0.002736,0.217254,0.250348,0.123441,0.009860,0.201770,0.042932,-1.114020,0,0.156037,0


In [9]:
train_0.to_csv(r"C:/Users/호재/Desktop/대학/3학년 2학기/기계학습론/data/IC_PBL/train_0.csv", encoding= 'cp949', index=False)
test_0.to_csv(r"C:/Users/호재/Desktop/대학/3학년 2학기/기계학습론/data/IC_PBL/test_0.csv", encoding= 'cp949', index=False)
train_25.to_csv(r"C:/Users/호재/Desktop/대학/3학년 2학기/기계학습론/data/IC_PBL/train_25.csv", encoding= 'cp949', index=False)
test_25.to_csv(r"C:/Users/호재/Desktop/대학/3학년 2학기/기계학습론/data/IC_PBL/test_25.csv", encoding= 'cp949', index=False)
train_50.to_csv(r"C:/Users/호재/Desktop/대학/3학년 2학기/기계학습론/data/IC_PBL/train_50.csv", encoding= 'cp949', index=False)
test_50.to_csv(r"C:/Users/호재/Desktop/대학/3학년 2학기/기계학습론/data/IC_PBL/test_50.csv", encoding= 'cp949', index=False)